# GetYourGuide Data Science Take Home Test

Author: Kai Chen

Date: Mar 2018

This notebook shows how I create a machine learning based method in order to predict the RPC (Revenue per Click).

train.csv and prediction.csv contain
- Date
- Keyword_ID
- Ad_group_ID
- Campaign_ID
- Account_ID
- Device_ID
- Match_type_ID

train.csv contains additional information
- Revenue
- Clicks
- Conversions

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline

from sklearn.feature_extraction import FeatureHasher
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score

import xgboost as xgb

## Step 1: Reading and Exploring the Data

* Define the file path

In [10]:
TRAIN_FILE = 'ds_dp_assessment/train.csv'
TEST_FILE = 'ds_dp_assessment/prediction.csv'

In [11]:
train_df = pd.read_csv(TRAIN_FILE)
display(train_df.head())

,Date,Keyword_ID,Ad_group_ID,Campaign_ID,Account_ID,Device_ID,Match_type_ID,Revenue,Clicks,Conversions
0,2014-12-14,88883763407,1042143654876,71672332180,861287123742,298643508640,95725474456,0.0,2,0
1,2014-12-14,837336088038,1038839743255,685307833020,654870334100,298643508640,95725474456,0.0,2,0
2,2014-12-14,220115136780,691864063258,551279817203,212779990172,848779586902,95725474456,0.0,2,0
3,2014-12-14,423389776473,1044670215658,551279817203,212779990172,848779586902,95725474456,0.0,5,0
4,2014-12-14,546746671314,355072204358,919168440848,212779990172,298643508640,95725474456,0.0,2,0


In [12]:
display(train_df.describe())

,Keyword_ID,Ad_group_ID,Campaign_ID,Account_ID,Device_ID,Match_type_ID,Revenue,Clicks,Conversions
count,8.285423e+06,8.285423e+06,8.285423e+06,8.285423e+06,8.285423e+06,8.285423e+06,8.285423e+06,8.285423e+06,8.285423e+06
mean,5.492429e+11,5.514791e+11,5.175203e+11,4.846436e+11,6.273406e+11,5.287287e+11,1.782111e+02,3.385137e+00,4.980832e-02
std,3.166589e+11,3.170199e+11,3.136106e+11,2.207485e+11,3.258393e+11,3.942597e+11,2.327171e+03,6.373065e+00,6.945033e-01
min,9.502900e+05,2.179346e+06,5.045747e+08,1.516649e+11,2.986435e+11,9.572547e+10,0.000000e+00,2.000000e+00,0.000000e+00
25%,2.738680e+11,2.764484e+11,2.463512e+11,2.213542e+11,2.986435e+11,9.572547e+10,0.000000e+00,2.000000e+00,0.000000e+00
50%,5.486864e+11,5.513241e+11,5.178672e+11,6.021828e+11,8.487796e+11,8.725446e+11,0.000000e+00,2.000000e+00,0.000000e+00
75%,8.217763e+11,8.263512e+11,7.850502e+11,6.049053e+11,8.487796e+11,8.944136e+11,0.000000e+00,2.000000e+00,0.000000e+00
max,1.099511e+12,1.099508e+12,1.099401e+12,9.814537e+11,1.077719e+12,8.944136e+11,1.029647e+06,1.827000e+03,1.180000e+03


In [13]:
display(train_df['Date'].unique())

array(['2014-12-14', '2014-12-15', '2014-12-16', '2014-12-17',
       '2014-12-18', '2014-12-19', '2014-12-20', '2014-12-21',
       '2014-12-22', '2014-12-23', '2014-12-24', '2014-12-25',
       '2014-12-26', '2014-12-27', '2014-12-28', '2014-12-29',
       '2014-12-30', '2014-12-31', '2015-01-01', '2015-01-02',
       '2015-01-03', '2015-01-04', '2015-01-05', '2015-01-06',
       '2015-01-07', '2015-01-08', '2015-01-09', '2015-01-10',
       '2015-01-11', '2015-01-12', '2015-01-13', '2015-01-14',
       '2015-01-15', '2015-01-16', '2015-01-17', '2015-01-18',
       '2015-01-19', '2015-01-20', '2015-01-21', '2015-01-22',
       '2015-01-23', '2015-01-24', '2015-01-25', '2015-01-26',
       '2015-01-27', '2015-01-28', '2015-01-29', '2015-01-30',
       '2015-01-31', '2015-02-01', '2015-02-02', '2015-02-03',
       '2015-02-04', '2015-02-05', '2015-02-06', '2015-02-07',
       '2015-02-08', '2015-02-09', '2015-02-10', '2015-02-11',
       '2015-02-12', '2015-02-13', '2015-02-14', '2015-

* Create a new column called 'RPC' (Revenue Per Click) which is our target variable

In [14]:
train_df['RPC'] = train_df['Revenue']/train_df['Clicks']
display(train_df.describe())

,Keyword_ID,Ad_group_ID,Campaign_ID,Account_ID,Device_ID,Match_type_ID,Revenue,Clicks,Conversions,RPC
count,8.285423e+06,8.285423e+06,8.285423e+06,8.285423e+06,8.285423e+06,8.285423e+06,8.285423e+06,8.285423e+06,8.285423e+06,8.285423e+06
mean,5.492429e+11,5.514791e+11,5.175203e+11,4.846436e+11,6.273406e+11,5.287287e+11,1.782111e+02,3.385137e+00,4.980832e-02,5.113287e+01
std,3.166589e+11,3.170199e+11,3.136106e+11,2.207485e+11,3.258393e+11,3.942597e+11,2.327171e+03,6.373065e+00,6.945033e-01,7.113045e+02
min,9.502900e+05,2.179346e+06,5.045747e+08,1.516649e+11,2.986435e+11,9.572547e+10,0.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00
25%,2.738680e+11,2.764484e+11,2.463512e+11,2.213542e+11,2.986435e+11,9.572547e+10,0.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00
50%,5.486864e+11,5.513241e+11,5.178672e+11,6.021828e+11,8.487796e+11,8.725446e+11,0.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00
75%,8.217763e+11,8.263512e+11,7.850502e+11,6.049053e+11,8.487796e+11,8.944136e+11,0.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00
max,1.099511e+12,1.099508e+12,1.099401e+12,9.814537e+11,1.077719e+12,8.944136e+11,1.029647e+06,1.827000e+03,1.180000e+03,5.148236e+05


In [16]:
display('max RPC {}'.format(train_df['RPC'].max()))
display('min RPC {}'.format(train_df['RPC'].min()))
display('mean RPC {}'.format(train_df['RPC'].mean()))
display('std RPC {}'.format(train_df['RPC'].std()))

'max RPC 514823.63'

'min RPC 0.0'

'mean RPC 51.13287018564376'

'std RPC 711.3045257230582'

* Show average RPC per feature

In [24]:
feature_column_names = ['Keyword_ID', 'Ad_group_ID', 'Campaign_ID', 'Account_ID', 'Device_ID', 'Match_type_ID']

In [28]:
# display(train_df.groupby('Keyword_ID')['RPC'].mean())
for feature_name in feature_column_names:
    display(train_df.groupby(feature_name)['RPC'].mean())

Keyword_ID
950290             0.000000
4191015            0.000000
6966484            0.000000
9220078            0.000000
12113917           0.000000
12584605           0.000000
14823863           0.000000
16612910           0.000000
17505300           0.000000
23879664           0.000000
26496419           0.000000
29907852           0.000000
30807963           0.000000
35327468         441.914107
36941707           0.000000
39981057           0.000000
41085538           0.000000
43463793           0.000000
45576139           0.584174
45697654           0.000000
50762704           0.000000
55006300           0.000000
55532314           0.000000
57039249           0.000000
57822915           0.000000
62752103           0.000000
63723074           0.000000
63824483           0.000000
66353044          11.066852
67227623           0.000000
                    ...    
1099455247644      0.000000
1099461799229      0.000000
1099462456677      0.000000
1099464229628      0.000000
109946455

Ad_group_ID
2179346             0.000000
7341258             0.000000
14545463            0.000000
15487922            0.000000
16641176            0.000000
17668908            0.000000
20324795            0.000000
23517389            0.000000
30750962            0.000000
31777912            0.000000
32111192            0.000000
32956576            0.000000
35346433            0.000000
38182905           70.551111
39789244            0.000000
40283961            0.000000
51918110            0.000000
54316010            0.000000
56388347            3.377040
61005096            0.000000
61919569            0.000000
64778580            0.000000
67455925          182.604167
71489966            0.000000
74385025            0.000000
75003051            0.000000
80354649            5.927391
81845734           82.022500
83758603          105.272549
88922141          206.234597
                    ...     
1099381693939       0.000000
1099381775019     858.495385
1099382761677     172.090648
10

Campaign_ID
504574657          0.000000
770750491        177.598277
934252170         43.007445
1023452889         0.000000
1112401400         0.000000
1473962740        82.758464
1851166350         0.000000
1956739518         0.000000
2472763673        20.769384
2533620164        61.640046
2938118854        48.305123
4411843497        39.318181
4676781606         0.000000
4869829574        29.428538
4925447727        59.553171
5234462229         0.000000
5235617114        21.557469
7721003771         0.000000
8171507359        24.974851
8348312607         0.000000
8380016917       410.268591
8737208207         0.000000
9591197160         0.000000
9612142282         0.000000
9628190609       129.940872
9853336086         0.000000
10645626434        0.000000
10916616189        0.000000
11412629023      259.393321
11801991809        0.000000
                    ...    
1088942159326     26.744793
1089508671861      0.000000
1090669898104     43.539414
1090803953279     28.306370
10909071

Account_ID
151664859558     36.170743
164144662657    109.072720
212779990172     66.104184
221354172146     35.988005
256188843610     10.144995
341124366337     67.722382
412971074791     21.007864
573604300663    116.606387
575525143937     35.597694
602182847798     40.534039
604905316813     48.403228
654870334100     53.328337
719583196582     69.602924
861287123742     82.834500
866124423689     47.650212
981453654147     97.761429
Name: RPC, dtype: float64

Device_ID
298643508640     65.941111
848779586902     30.608125
1077718730738    49.576451
Name: RPC, dtype: float64

Match_type_ID
95725474456     60.961075
872544605608    41.741624
894413617560    43.553838
Name: RPC, dtype: float64

It shows that all the feature affect the RPC. However, most of the keywords do not affect RPC.

* Show average RPC per two-features combination

In [33]:
for feature_name1 in feature_column_names:
    for feature_name2 in feature_column_names:
        if feature_name1 != feature_name2:
            display(train_df.groupby([feature_name1, feature_name2])['RPC'].mean())

Keyword_ID     Ad_group_ID  
950290         511252471042       0.000000
               740677272847       0.000000
4191015        463913492572       0.000000
6966484        850348036447       0.000000
               998575201817       0.000000
9220078        529821357260       0.000000
12113917       61088338981        0.000000
               757834154119       0.000000
               799685625223       0.000000
12584605       524892551135       0.000000
14823863       259641772696       0.000000
               435429699636       0.000000
               673988188593       0.000000
16612910       297740430388       0.000000
               806264023256       0.000000
17505300       704127859151       0.000000
23879664       524442584034       0.000000
26496419       1066513998560      0.000000
29907852       513414133600       0.000000
               559320120978       0.000000
30807963       146380677095       0.000000
               218975026360       0.000000
               3748485734

Keyword_ID     Campaign_ID  
950290         352608953894       0.000000
               393664657081       0.000000
4191015        597530476195       0.000000
6966484        184219020943       0.000000
               806679371207       0.000000
9220078        1014454260385      0.000000
12113917       217158967869       0.000000
               697656312616       0.000000
12584605       20237246307        0.000000
14823863       189337550548       0.000000
               330204765951       0.000000
               1095028050196      0.000000
16612910       447834681534       0.000000
               958903719756       0.000000
17505300       4869829574         0.000000
23879664       184219020943       0.000000
26496419       926423156625       0.000000
29907852       597530476195       0.000000
30807963       444539244775       0.000000
               647347436254       0.000000
               933697468832       0.000000
               967176606161       0.000000
35327468       1707377047

Keyword_ID     Account_ID  
950290         654870334100      0.000000
4191015        654870334100      0.000000
6966484        221354172146      0.000000
9220078        221354172146      0.000000
12113917       604905316813      0.000000
               861287123742      0.000000
12584605       604905316813      0.000000
14823863       602182847798      0.000000
               604905316813      0.000000
16612910       604905316813      0.000000
17505300       221354172146      0.000000
23879664       221354172146      0.000000
26496419       256188843610      0.000000
29907852       654870334100      0.000000
30807963       221354172146      0.000000
               412971074791      0.000000
35327468       866124423689    441.914107
36941707       604905316813      0.000000
               719583196582      0.000000
               861287123742      0.000000
39981057       212779990172      0.000000
41085538       151664859558      0.000000
43463793       604905316813      0.000000
455761

Keyword_ID     Device_ID    
950290         298643508640       0.000000
               848779586902       0.000000
               1077718730738      0.000000
4191015        298643508640       0.000000
               848779586902       0.000000
               1077718730738      0.000000
6966484        298643508640       0.000000
               1077718730738      0.000000
9220078        298643508640       0.000000
12113917       298643508640       0.000000
               848779586902       0.000000
12584605       298643508640       0.000000
14823863       298643508640       0.000000
               1077718730738      0.000000
16612910       1077718730738      0.000000
17505300       298643508640       0.000000
23879664       298643508640       0.000000
26496419       298643508640       0.000000
29907852       298643508640       0.000000
               848779586902       0.000000
               1077718730738      0.000000
30807963       298643508640       0.000000
               8487795869

Keyword_ID     Match_type_ID
950290         894413617560       0.000000
4191015        894413617560       0.000000
6966484        95725474456        0.000000
9220078        872544605608       0.000000
12113917       894413617560       0.000000
12584605       95725474456        0.000000
14823863       872544605608       0.000000
16612910       894413617560       0.000000
17505300       894413617560       0.000000
23879664       894413617560       0.000000
26496419       894413617560       0.000000
29907852       894413617560       0.000000
30807963       872544605608       0.000000
35327468       95725474456      441.914107
36941707       872544605608       0.000000
39981057       95725474456        0.000000
41085538       894413617560       0.000000
43463793       95725474456        0.000000
45576139       894413617560       0.584174
45697654       894413617560       0.000000
50762704       872544605608       0.000000
55006300       894413617560       0.000000
55532314       8944136175

...

Match_type_ID  Keyword_ID   
95725474456    6966484            0.000000
               12584605           0.000000
               35327468         441.914107
               39981057           0.000000
               43463793           0.000000
               70855622           0.000000
               73275912           0.000000
               78045300           0.000000
               86110178           0.000000
               87057116           0.000000
               95890338           0.000000
               97301035           0.000000
               101164445          0.000000
               104558956          0.000000
               106111244          0.000000
               114974125          0.000000
               119139423          0.000000
               122207191          0.000000
               122896109          0.000000
               126983678          0.000000
               129302786          0.000000
               146418075          0.000000
               147048135 

Match_type_ID  Ad_group_ID  
95725474456    2179346             0.000000
               7341258             0.000000
               16641176            0.000000
               17668908            0.000000
               20324795            0.000000
               23517389            0.000000
               30750962            0.000000
               31777912            0.000000
               32111192            0.000000
               35346433            0.000000
               38182905            0.000000
               51918110            0.000000
               56388347            0.000000
               61919569            0.000000
               64778580            0.000000
               67455925            0.000000
               74385025            0.000000
               80354649            0.000000
               83758603           46.996796
               88922141          431.448171
               92594891            0.000000
               98172110          110.725876
   

Match_type_ID  Campaign_ID  
95725474456    504574657          0.000000
               770750491        215.059783
               934252170         44.322555
               1023452889         0.000000
               1112401400         0.000000
               1473962740       102.146958
               1851166350         0.000000
               1956739518         0.000000
               2472763673        46.380275
               2533620164        31.441290
               2938118854        38.648505
               4411843497        24.272971
               4676781606         0.000000
               4869829574        31.356738
               4925447727        41.111023
               5234462229         0.000000
               5235617114         7.487607
               8171507359        49.215147
               8348312607         0.000000
               8380016917       414.228717
               9591197160         0.000000
               9612142282         0.000000
               9628190609

Match_type_ID  Account_ID  
95725474456    151664859558     38.182534
               164144662657    120.248456
               212779990172     73.678013
               221354172146     48.606668
               256188843610     11.179141
               341124366337     73.710073
               412971074791     27.652613
               573604300663    125.019315
               575525143937     31.987930
               602182847798     47.180329
               604905316813     59.559798
               654870334100     65.130490
               719583196582     80.696076
               861287123742     91.736908
               866124423689     46.338901
               981453654147    117.239617
872544605608   151664859558     33.677874
               164144662657     93.208201
               212779990172     54.661843
               221354172146     29.471977
               256188843610      8.453470
               341124366337     60.771996
               412971074791     16.666693
      

Match_type_ID  Device_ID    
95725474456    298643508640     80.501984
               848779586902     36.102368
               1077718730738    58.165941
872544605608   298643508640     53.096479
               848779586902     25.037566
               1077718730738    40.308759
894413617560   298643508640     55.269435
               848779586902     26.099082
               1077718730738    42.919986
Name: RPC, dtype: float64

In [36]:
# load test data
test_df = pd.read_csv(TEST_FILE)

## Step 2: Preparing datasets for building machine learning model

In [37]:
# prepare datasets: x_train (features of train set), y_train (targets of train set), and x_test (features of test set)
feature_column_names = ['Keyword_ID', 'Ad_group_ID', 'Campaign_ID', 'Account_ID', 'Device_ID', 'Match_type_ID']

X_train = train_df[feature_column_names]
Y_train = train_df['RPC']

X_test = test_df[feature_column_names]

display('x train shape')
display(X_train.shape)
display('y train shape')
display(Y_train.shape)
display('x test shape')
display(X_test.shape)

'x train shape'

(8285423, 6)

'y train shape'

(8285423,)

'x test shape'

(578012, 6)

## Step 3: Feature engineering

TODO: use more sophisticated category feature engineering approaches

- References

    - https://blog.myyellowroad.com/using-categorical-data-in-machine-learning-with-python-from-dummy-variables-to-deep-category-66041f734512
    - https://medium.com/unstructured/how-feature-engineering-can-help-you-do-well-in-a-kaggle-competition-part-i-9cc9a883514d


In [20]:
# hasher = FeatureHasher(n_features=5,
#             non_negative=True,
#             input_type='string')
# ## Keyword_ID
# tmp_features = hasher.transform(X_train['Keyword_ID'].astype(str).tolist()).toarray()
# for i in range(5):
#     X_train['keyword_'+str(i)] = tmp_features[:, i]
# X_train.drop(['Keyword_ID'], axis=1, inplace=True)
# print(X_train.describe())

/Users/kai/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)


## Step 4: Model training

### prepare datasets for xgboost

In [41]:
# prepare datasets for xgb
X_train_sub, X_val, Y_train_sub, Y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=42)

dtrain_mat = xgb.DMatrix(X_train, Y_train)
dtrain_sub_mat = xgb.DMatrix(X_train_sub, Y_train_sub)
dval_mat = xgb.DMatrix(X_val, Y_val)
dtest_mat = xgb.DMatrix(X_test)

### initialize xgboost parameters

In [43]:
# Initialize xgb parameters
# details of xgboost parameters can be found in
# http://xgboost.readthedocs.io/en/latest/parameter.html
xgb_params = {'eta':0.1,
              'seed':42,
              'gamma':0,
              'subsample':0.8,
              'colsample_bytree':0.8,
              'objective':'reg:linear',
              #'objective':'binary:logistic',
              'max_depth': 8,
              'min_child_weight': 1,
              #'metrics':['auc'],
              #'metrics':['mae'],
              'metrics':['rmse'],
              'eval_metric':['rmse'],
              'nthread': 8,
              'n_fold': 2,
              # 'n_jobs': 4,
              'scale_pos_weight': 1,
              'num_boost_round': 200,
              # 'n_estimators':200,
              'early_stopping_rounds': 10,
              }

### Hyperparameter tuning with k-fold cross validation

- The ideas are taken from https://cambridgespark.com/content/tutorials/hyperparameter-tuning-in-xgboost/index.html

#### 'max_depth' and 'min_child_weight'


In [ ]:
# tune 'max_depth' and 'min_child_weight'
print('tune max_depth and min_child_weight ...')
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(4, 7, 1)
    for min_child_weight in range(4, 7, 1)
]

min_rmse = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(max_depth, min_child_weight))

    xgb_params['max_depth'] = max_depth
    xgb_params['min_child_weight'] = min_child_weight

    # TODO: change metrics
    cv_results = xgb.cv(
        xgb_params,
        dtrain_sub_mat,
        num_boost_round=xgb_params['num_boost_round'],
        seed=42,
        nfold=xgb_params['n_fold'],
        # metrics={'mae'},
        metrics={'rmse'},
        early_stopping_rounds=xgb_params['early_stopping_rounds']
    )

    # Update best score
    mean_rmse = cv_results['test-rmse-mean'].min()
    boost_rounds = cv_results['test-rmse-mean'].argmin()
    print("\tRMSE {} for {} rounds".format(mean_rmse, boost_rounds))
    if mean_rmse < min_rmse:
        min_rmse = mean_rmse
        best_params = (max_depth, min_child_weight)

print("Best params: {}, {}, RMSE: {}".format(best_params[0], best_params[1], min_rmse))

xgb_params['max_depth'] = best_params[0]
xgb_params['min_child_weight'] = best_params[1]

#### 'subsample' and 'colsample'
- 'subsample' corresponds the fraction of observations (the rows) to subsample at each step. By default it is set to 1 meaning that we use all rows. 
- 'colsample_bytree' corresponds to the fraction of features (the columns) to use. By default it is set to 1 meaning that we will use all features.

In [ ]:
gridsearch_params = [
        (subsample, colsample)
        for subsample in [i/10. for i in range(6, 11, 2)]
        for colsample in [i/10. for i in range(6, 11, 2)]
    ]

min_rmse = float("Inf")
best_params = None

for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))

    xgb_params['subsample'] = subsample
    xgb_params['colsample_bytree'] = colsample

    cv_results = xgb.cv(
        xgb_params,
        dtrain_sub_mat,
        num_boost_round=xgb_params['num_boost_round'],
        seed=42,
        nfold=xgb_params['n_fold'],
        # metrics={'mae'},
        metrics={'rmse'},
        early_stopping_rounds=xgb_params['early_stopping_rounds']
    )

    mean_rmse = cv_results['test-rmse-mean'].min()
    boost_rounds = cv_results['test-rmse-mean'].argmin()
    print("\tRMSE {} for {} rounds".format(mean_rmse, boost_rounds))
    if mean_rmse < min_rmse:
        min_rmse = mean_rmse
        best_params = (subsample, colsample)

print("Best params: {}, {}, RMSE: {}".format(best_params[0], best_params[1], mean_rmse))

xgb_params['subsample'] = best_params[0]
xgb_params['colsample_bytree'] = best_params[1]

#### learning rate

In [ ]:
print('tune learning rate ...')

min_rmse = float("Inf")
best_learning_rate = None
# learning_rate_range = [0.1, 0.05, 0.01, 0.005]
learning_rate_range = [0.1, 0.05, 0.01]

for eta in learning_rate_range:
    print("CV with eta={}".format(eta))

    xgb_params['eta'] = eta

    cv_results = xgb.cv(
            xgb_params,
            dtrain_sub_mat,
            num_boost_round=xgb_params['num_boost_round'],
            seed=42,
            nfold=xgb_params['n_fold'],
            # metrics=['mae'],
            metrics={'rmse'},
            early_stopping_rounds=xgb_params['early_stopping_rounds']
          )

    mean_rmse = cv_results['test-rmse-mean'].min()
    boost_rounds = cv_results['test-rmse-mean'].argmin()
    print("\tRMSE {} for {} rounds".format(mean_rmse, boost_rounds))
    if mean_rmse < min_rmse:
        min_rmse = mean_rmse
        best_learning_rate = eta

print("Best params: {}, RMSE: {}".format(best_learning_rate, mean_rmse))

xgb_params['eta'] = best_learning_rate

#### number of boost round

In [ ]:
print('find the best number of boost ...')
xgb_clf = xgb.train(
    xgb_params,
    dtrain_sub_mat,
    num_boost_round=xgb_params['num_boost_round'],
    evals=[(dval_mat, "val")],
    early_stopping_rounds=xgb_params['early_stopping_rounds']
)
xgb_params['num_boost_round'] = xgb_clf.best_iteration + 1

In [ ]:
# show parameters
print('XBG parameters')
print(xgb_params)

### Model training

- without early stopping

In [ ]:
# without early stopping
# print('train a xgb model ...')
# xgb_clf = xgb.train(
#     xgb_params,
#     dtrain_mat,
#     num_boost_round=xgb_params['num_boost_round'],
# )

- with early stopping

In [ ]:
# with early stopping
print('train a xgb model ...')
xgb_clf = xgb.train(
        xgb_params,
        dtrain_sub_mat,
        num_boost_round=xgb_params['num_boost_round'],
        evals=[(dval_mat, "val")],
        early_stopping_rounds=xgb_params['early_stopping_rounds'])

- save the model

In [ ]:
xgb_model_path = "xgb-[n_fold]{}-[n_estimators]{}-[max_depth]{}-[min_child_weight]{}-[eta]{}.model".format(xgb_params['n_fold'],
                                                                                                           xgb_params['num_boost_round'],
                                                                                                           xgb_params['max_depth'],
                                                                                                           xgb_params['min_child_weight'],
                                                                                                           xgb_params['eta'])

xgb_clf.save_model(xgb_model_path)
# joblib.dump(xgb_clf, xgb_model_path)
print('save xgb model to {}'.format(xgb_model_path))

# load the model
# xgb_clf = xgb.Booster()
# xgb_clf.load_model(xgb_model_path)

## Step 5: Make Predictions

In [ ]:
# predict RPC of the test data
y_pred = xgb_clf.predict(dtest_mat)

# post-processing
for i, pred in enumerate(y_pred):
    if pred < 0:
        y_pred[i] = 0
# print(y_pred)

# save the predictions
y_pred_df = pd.DataFrame(data=y_pred)
pred_path = "pred-[xgb]-[n_fold]{}-[n_estimators]{}-[max_depth]{}-[min_child_weight]{}-[eta]{}.csv".format(xgb_params['n_fold'],
                                                                                                           xgb_params['num_boost_round'],
                                                                                                           xgb_params['max_depth'],
                                                                                                           xgb_params['min_child_weight'],
                                                                                                           xgb_params['eta'])
y_pred_df.to_csv(pred_path, index=False)
print('save prediction to {}'.format(pred_path))

## Future work

- Develop more sophisticated category feature engineering approaches.
- Blend different regression models.
- Evaluate the model performance and compare it with the baseline methods.
- Study other solutions on kaggle for relevant the competitions (e.g., Click-Through Rate Prediction by Avazu, Display Advertising Challenge, ). Some relevant articles are:
    - https://www.csie.ntu.edu.tw/~r01922136/slides/kaggle-avazu.pdf
    - https://www.quantmetry.com/single-post/2015/03/12/My-first-Kaggle-challenge-the-Avazu-CTR-contest
    - https://medium.com/@chris_bour/our-avazu-kaggle-challenge-logbook-b522ccb1ed21

